In [13]:
# Plots the National Power Outage data on a map in the North East. 

# Uses google maps data, national flood watch, weather data and power outage data.

# Needs API keys of:
# api_key = 'GOOGLE_MAPS_API_KEY' # https://developers.google.com/maps/documentation/javascript/get-api-key
# api_key = "WEATHER_API_KEY"     # https://www.weatherapi.com/

import folium
import requests
import pandas as pd

Mapping Durham

In [14]:
import json

def mapify_durham(map = None):
    """ creates a map of Durham"""
    import folium
    with open('durham.geojson') as f:
        durham = json.load(f)
    list_durham = durham["geometry"]["coordinates"][0][0]

    durham_geom = [[i[1], i[0]] for i in list_durham]
    centre_coods = [54.7753, -1.5849]
    if map:
        map = map
    else:
        map = folium.Map(location=centre_coods, zoom_start=10) #New Castle
    folium.Polygon(
        locations = durham_geom,
        color='blue', 
        fill=True, 
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(map)
    map
    return map
mapify_durham() # Function call

Mapping any place using google's API.

In [15]:
import requests
import folium 
import time
import json

def map_this(query, key, map = None):
    """
    Retrieves a list of queried points in a given location.

    Args:
        query (str): The location to search for hospitals.

    Returns:
        list: A list of hospital names and addresses.
    """
    url = 'https://maps.googleapis.com/maps/api/place/textsearch/json'
    # key = os.environ.get('GOOGLE_MAPS_API_KEY')
    
    if not key:
        raise ValueError('Google Maps API key not found.')

    params = {
        'query': query,
        'key': key,
    }

    stored = []
    while True:
        try:
            res = requests.get(url, params=params)
            res.raise_for_status()
            data = res.json()
            stored.extend(data['results'])
            # hospitals.extend(data)
            if 'next_page_token' in data:
                params['pagetoken'] = data['next_page_token']
                time.sleep(2)
            else:
                break
        except:
            "Error"
    if map:
        map = map
    else:
        map = folium.Map(location=[54.9783, -1.6178], zoom_start=12) #New Castle
    for i in range(0,len(stored)):
        cc = [stored[i]["geometry"]["location"]["lat"], stored[i]["geometry"]["location"]["lng"]]
        folium.Marker(
                    cc
                    , icon=folium.Icon(icon ='fa-h-square', prefix='fa', color='red')
                    , tooltip = [stored[i]["name"]] 
                ).add_to(map)
    return map

hospitals = map_this('all+hospitals+in+north+east+england')

hospitals

Getting weather/temperature data

In [27]:
# Adding the weather data

import requests
def map_weather(map = None):
    import folium
    api_key = "WEATHER_API_KEY"
    cities = ['Bishop Auckland', 'Durham', 'Barnard Castle', 'Stanhope', 'Peterlee']
    country = 'UK'
    # Create map centered on Newcastle
    # map = folium.Map(location=[-32.9283, 151.7817], zoom_start=10)
    if map:
        map = map
    else:
        map = folium.Map(location=[54.9783, -1.6178], zoom_start=12)
    
    for city in cities:
        url = f'https://api.weatherapi.com/v1/current.json?key={api_key}&q={city},{country}'
        response = requests.get(url)
        data = response.json()
        temperature = data['current']['temp_c']
        wind_speed = data['current']['wind_kph']
        humidity = data['current']['humidity']
        pressure = data['current']['pressure_mb']
        # folium.Marker(location=get_coordinates(city, country), 
        folium.Marker(location= [data["location"]["lat"], data["location"]["lon"]],
                    popup=f'Temperature: {temperature}°C\nWind speed: {wind_speed} kph\nHumidity: {humidity}%\nPressure: {pressure} mb', 
                    icon=folium.Icon(icon = "fa-solid fa-wind", prefix = "fa", color='blue')).add_to(map)
    return map
cities = ['Bishop Auckland', 'Durham', 'Barnard Castle', 'Stanhope', 'Peterlee']


Obtaining powercuts from the Northen power grid website, - Color is the nature of the fault. 

In [17]:
import requests
import pandas as pd
import json
import folium

def get_power_cuts(map=None):
    url = 'https://power.northernpowergrid.com/Powercut_API/rest/powercuts/getall'
    response = requests.get(url)
    data = response.json()
    new = pd.DataFrame(data[0:len(data)][0:len(data)])
    # Create a map object
    if map:
        pass
    else:
        map = folium.Map(location=[54.9783, -1.6178], zoom_start=12)
    # # Add a marker for each row of the dataframe
    for index, row in new.iterrows():
        color = ""
        if row['Area']== 'North East':
            if row["NatureOfOutage"] == 'Localised Fault':
                color = 'red'
            elif row["NatureOfOutage"] == 'Large Area Fault':
                color = 'blue'
            else:
                color = 'green'
            folium.Marker([float(row['Lat']), float(row['Lng'])], popup=row[['NatureOfOutage']].to_dict(),
                        icon=folium.Icon(icon = 'fa-solid fa-bolt',prefix = 'fa' , color=color)).add_to(map)
    return map
# # Display the map
get_power_cuts()

Checking for floods

In [18]:

def get_floods(floods):
    url = 'https://check-for-flooding.service.gov.uk/api/warnings.geojson'
    response = requests.get(url)
    data = response.json()
    # floods = folium.Map(location=[54.9783, -1.6178], zoom_start=12) #New Castle
    # ADDING MARKERS
    for i in range(0,len(data["features"])):
        cc = [data["features"][i]["geometry"]["coordinates"][1], data["features"][i]["geometry"]["coordinates"][0]]
        if data["features"][i]["properties"]["severity_value"] == 1:
            color = 'green'
        elif data["features"][i]["properties"]["severity_value"] == 2:
            color = 'yellow'
        elif data["features"][i]["properties"]["severity_value"] == 3:
            color = 'orange'
        else:
            color = 'red'
        folium.Marker(
                    cc
                    , icon=folium.Icon(icon = "fa-solid fa-water", prefix = "fa", color=color)
                    , tooltip = [data["features"][i]["properties"]["severity"],data["features"][i]["properties"]["severity_value"], data["features"][i]["properties"]["ta_name"]] 
            
                ).add_to(floods)
    return floods

map = folium.Map(location=[54.9783, -1.6178], zoom_start=12) #New Castle
get_floods(map)

Combining all functions on a single map

In [25]:
key = "Enter your Google Maps API key here"
mapped_locations = map_this("All hospitals in North East England", key)
mapped_locations 
flood_info = get_floods(mapped_locations)
power_cut_info = get_power_cuts(flood_info)
weather_info = map_weather(power_cut_info)
durham_map = mapify_durham(weather_info)
durham_map

Demonstrating the opportunity of querying other vulnerable centres.

### Future work 
Add code that flares/flashes/becomes hotter if the power cut coordinate is near that of a vulnerable centre.
Extend the code to produce a live heat map of Risk (needs Generalised linear mixed models/ KDEs) using various combined

####

In [28]:
mapped_locations = map_this("All old age homes in North East England")
mapped_locations 
flood_info = get_floods(mapped_locations)
power_cut_info = get_power_cuts(flood_info)
weather_info = map_weather(power_cut_info)
durham_map = mapify_durham(weather_info)
durham_map